In [1]:
import scipy.stats as stats
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

import bokeh.charts as charts
charts.output_notebook()

Loading BokehJS ...

### Part 1

##### 1)

The value of $\mu$ does not affect the *LIP* because

$$Y = e^{\mu+\sigma z}$$
$$\hphantom Y = e^{\mu}e^{\sigma Z}$$

Hence, each random varible $Y$ is simply multiplied by a constant. For both $N(0,1)$ and the log-normal distribution,

$$\text{median} = \textbf{E}(Y)$$

If the median $m = \textbf{E}(Y)$, then $\textbf{E}(cY) = c\textbf{E}(Y) = cm$. Because the *LIP* is a fraction of the median, it is not affected by the value of $\mu$.

##### 2)

It has been established that $\mu$ does not affect the *LIP*, so let's assume $\mu = 0$. The median of a log-normal distribution with $\mu = 0$ is 1 (because for $N(0,1)$ $\mu = 0$). The half-median is therefore 0.5. This is confirmed as follows.

In [2]:
#s=0.6 is used here, but the result will be the same for all values of s > 0
median = stats.lognorm.ppf(0.5, s=0.6)
half_median = median / 2
half_median

0.5

The $LIP(0.5,0.5)$ is the proportion of incomes below the half-median.

In [3]:
LIP = stats.lognorm.cdf(half_median, s=0.6)
LIP

0.12399499425286353

##### 3)

In [4]:
def simulation(n):
    #x = np.random.lognormal(0, 0.6, n)
    x = stats.lognorm.rvs(s=0.6, loc=0, size=n)
    half_median = np.median(x)/2
    return (x < half_median).mean()

In [5]:
iterations = 400000

results = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(simulation)(1001) for i in range(0, iterations))
results = np.array(results)
results[:5]

array([ 0.11488511,  0.11488511,  0.11488511,  0.11488511,  0.12287712])

In [6]:
mean = results.mean()
mean

0.12391027972027972

In [7]:
variance = results.var()
variance

0.0001105418168683464

$\textbf{P}(Y < 0.75 \times LIP(0.5,0.5)) =$

In [8]:
(results < 0.75*LIP).mean()

0.0013600000000000001

$\textbf{P}(Y > 1.25 \times LIP(0.5,0.5)) =$

In [9]:
(results > 1.25*LIP).mean()

0.0017474999999999999

A histogram of the simulated LIP values is given below. As expected (per Normal approximation to Binomial distribution), the simulated LIP values are normally distributed.

In [10]:
bins = 30
x = np.linspace(results.min(), results.max(), num=bins*100)
fitted_loc, fitted_s = stats.norm.fit(results)
fitted = stats.norm.pdf(x=x, loc=fitted_loc, scale=fitted_s)

p = charts.Histogram(results, bins=bins, title="LIP(0.5,0.5) Histogram", density=True)
p.line(x, fitted, line_width=3, legend="Fitted Normal Distribution")
p.xaxis.axis_label = "LIP values"
p.yaxis.axis_label = "Relative frequencies"
charts.show(p)